Install and import library modules. Download English stopwords and WordNet. Set logging to INFO

In [1]:
import sys
!{sys.executable} -m pip install gensim
!{sys.executable} -m pip install pyLDAvis
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install nltk

import re, gensim, pyLDAvis, logging, datetime, multiprocessing, os, pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import floor, ceil
from nltk.corpus import stopwords
from nltk import download
from nltk.stem import WordNetLemmatizer

download('stopwords')
download('wordnet')

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.1.2/libexec/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.1.2/libexec/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.1.2/libexec/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.1.2/libexec/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.1.2/libexec/bin/python3.8 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/2.1.2/libexec/bin/python3.8 -m pip install --upgrade pip' command.


[nltk_data] Downloading package stopwords to /Users/joost/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /Users/joost/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Read the data as pandas dataframe, remove items with missing abstracts, and fill other missing data with empty strings

In [2]:
df = pd.read_csv('data/data_merged_and_filtered.csv')
df = df[df['abstract'].notna()]
df.fillna("", inplace=True)

Combine the text fiels into one

In [3]:
df["data"] = df["abstract"] + " " + df["keywords"] + " " + df["title"]

2021-07-18 15:02:44,439 : INFO : NumExpr defaulting to 8 threads.


Remove copyright notices from the text

In [4]:
copyright_re = "(©|&#x00A9;|Copyright)(.*?(?=B\\.V\\.|Ltd|\\.))(B\\.V\\.|Ltd|\\.)"
rights = "All rights reserved."
df.replace(copyright_re, '', regex=True, inplace=True)
df.replace(rights, '', regex=False, inplace=True)

Tokenize, convert to lowercase, remove punctuation, remove stop-words, lemmatize

In [5]:
def preprocess(text):
  result = [word for word in gensim.utils.simple_preprocess(text, deacc=True) if word not in stop_words]
  wordnet_lemmatizer = WordNetLemmatizer()
  result = [wordnet_lemmatizer.lemmatize(word) for word in result]
  return result

stop_words = stopwords.words('english')
df["data_preprocessed"] = df["data"].map(preprocess)

Generate the Dictionairy and perform relative pruning

In [6]:
dictionary = gensim.corpora.Dictionary(df["data_preprocessed"])
dictionary.filter_extremes(no_below=(floor(0.005 * len(df))), no_above=0.99)

2021-07-18 15:07:48,294 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-07-18 15:07:50,678 : INFO : adding document #10000 to Dictionary(27074 unique tokens: ['ability', 'acceptable', 'additionally', 'analysis', 'anti']...)
2021-07-18 15:07:52,964 : INFO : adding document #20000 to Dictionary(38089 unique tokens: ['ability', 'acceptable', 'additionally', 'analysis', 'anti']...)
2021-07-18 15:07:55,071 : INFO : adding document #30000 to Dictionary(46717 unique tokens: ['ability', 'acceptable', 'additionally', 'analysis', 'anti']...)
2021-07-18 15:07:57,078 : INFO : adding document #40000 to Dictionary(53696 unique tokens: ['ability', 'acceptable', 'additionally', 'analysis', 'anti']...)
2021-07-18 15:07:59,091 : INFO : adding document #50000 to Dictionary(59648 unique tokens: ['ability', 'acceptable', 'additionally', 'analysis', 'anti']...)
2021-07-18 15:08:01,086 : INFO : adding document #60000 to Dictionary(64997 unique tokens: ['ability', 'acceptable', 'additional

Make the bag of words

In [7]:
corpus = [dictionary.doc2bow(doc) for doc in df["data_preprocessed"]]

Generate all models for evaluation

In [ ]:
k = [1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361] #later added [30, 40, 42]
random_states = [24, 1024] #later added [27, 1027]

texts = df["data_preprocessed"]
chunksize = ceil(len(texts))
passes = 20
iterations = 100
eval_every = None
dir_prior = 'auto'

for random_state in random_states:
    for num_topics in k:
        
        name = "rs{}k{}".format(random_state, num_topics)
        
        
        if 'model_' + name in os.listdir('models'):
            print(datetime.datetime.now().strftime('%H:%M:%S') + ' Model ' + name + ' already in directory. Skipped')
        
        else:
            print(datetime.datetime.now().strftime('%H:%M:%S') + " Building model " + name)
            lda_model = gensim.models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, alpha=dir_prior, eta=dir_prior, random_state=random_state, chunksize=chunksize, passes=passes, iterations=iterations)

            lda_model.save(os.path.join('models', 'model_' + name))

2021-07-18 15:08:17,392 : INFO : using autotuned alpha, starting with [0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.00390625, 0.003

15:08:17 Model rs24k1 already in directory. Skipped
15:08:17 Model rs24k4 already in directory. Skipped
15:08:17 Model rs24k9 already in directory. Skipped
15:08:17 Model rs24k16 already in directory. Skipped
15:08:17 Model rs24k25 already in directory. Skipped
15:08:17 Model rs24k36 already in directory. Skipped
15:08:17 Model rs24k49 already in directory. Skipped
15:08:17 Model rs24k64 already in directory. Skipped
15:08:17 Model rs24k81 already in directory. Skipped
15:08:17 Model rs24k100 already in directory. Skipped
15:08:17 Model rs24k121 already in directory. Skipped
15:08:17 Model rs24k144 already in directory. Skipped
15:08:17 Model rs24k169 already in directory. Skipped
15:08:17 Model rs24k196 already in directory. Skipped
15:08:17 Model rs24k225 already in directory. Skipped
15:08:17 Building model rs24k256


2021-07-18 15:08:17,616 : INFO : running online (multi-pass) LDA training, 256 topics, 20 passes over the supplied corpus of 79433 documents, updating model once every 79433 documents, evaluating perplexity every 79433 documents, iterating 100x with a convergence threshold of 0.001000


Evaluate all models present in the current directory

In [ ]:
models_to_evaluate = []

for entry in os.scandir('models'):
    if entry.path.endswith('.state'):
        models_to_evaluate.append(entry.path[:-6])
        
for path in models_to_evaluate:
    model = gensim.models.LdaModel.load(path)
    coherence_model_lda = gensim.models.coherencemodel.CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence = coherence_model_lda.get_coherence()
        
    with open(os.path.join('models', 'scores.txt'), "a") as f:
        f.write("\"" + str(os.path.split(path)[1]) + "\"" ' : ' + str(coherence) + ',\n')   

Save the preprocessed data, corpus and dictionary for later exploration  

In [ ]:
df.to_csv('data/data_for_interpretation.csv')

with open('models/corpus.pkl', 'wb') as f:
    pickle.dump(corpus, f)
    
dictionary.save('models/dictionary')